In [1]:
import pandas as pd
from sqlalchemy import create_engine
import time
import math

In [2]:
# Chemin vers ton fichier
parquet_file = "../yellow_tripdata_2025-01.parquet"

# Lire le fichier dans un DataFrame pandas
df = pd.read_parquet(parquet_file)

# Afficher les premières lignes pour vérifier
df.head()


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,1,2025-01-01 00:18:38,2025-01-01 00:26:59,1.0,1.60,1.0,N,229,237,1,10.0,3.5,0.5,3.00,0.0,1.0,18.00,2.5,0.0,0.0
1,1,2025-01-01 00:32:40,2025-01-01 00:35:13,1.0,0.50,1.0,N,236,237,1,5.1,3.5,0.5,2.02,0.0,1.0,12.12,2.5,0.0,0.0
2,1,2025-01-01 00:44:04,2025-01-01 00:46:01,1.0,0.60,1.0,N,141,141,1,5.1,3.5,0.5,2.00,0.0,1.0,12.10,2.5,0.0,0.0
3,2,2025-01-01 00:14:27,2025-01-01 00:20:01,3.0,0.52,1.0,N,244,244,2,7.2,1.0,0.5,0.00,0.0,1.0,9.70,0.0,0.0,0.0
4,2,2025-01-01 00:21:34,2025-01-01 00:25:06,3.0,0.66,1.0,N,244,116,2,5.8,1.0,0.5,0.00,0.0,1.0,8.30,0.0,0.0,0.0


In [3]:
# -----------------------------
# Étape 3 : Nettoyer/adapter les noms de colonnes
# -----------------------------
df.columns = [c.lower().replace(' ', '_') for c in df.columns]


In [4]:
# -----------------------------
# Étape 4 : Conversion des dates
# -----------------------------
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

In [5]:
# -----------------------------
# Étape 5 : Créer la connexion PostgreSQL
# -----------------------------
user = "root"
password = "root"
host = "localhost"
port = "5432"
database = "ny_taxi"

engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")


In [6]:
# -----------------------------
# Étape 6 : Charger les données dans PostgreSQL
# -----------------------------
table_name = "yellow_tripdata_2025_01"
chunksize = 100000

# Drop and create fresh table with first chunk (so schema is set up)
df.iloc[:0].to_sql(table_name, engine, if_exists="replace", index=False)

# Insert in chunks with elapsed time logging
n_chunks = math.ceil(len(df) / chunksize)

for i in range(n_chunks):
    start = time.time()
    chunk = df.iloc[i*chunksize : (i+1)*chunksize]
    chunk.to_sql(table_name, engine, if_exists="append", index=False)
    elapsed = time.time() - start
    print(f"Chunk {i+1}/{n_chunks} inserted ({len(chunk)} rows) in {elapsed:.2f} seconds")

Chunk 1/348 inserted (10000 rows) in 0.82 seconds
Chunk 2/348 inserted (10000 rows) in 0.57 seconds
Chunk 3/348 inserted (10000 rows) in 0.57 seconds
Chunk 4/348 inserted (10000 rows) in 0.55 seconds
Chunk 5/348 inserted (10000 rows) in 0.60 seconds
Chunk 6/348 inserted (10000 rows) in 0.55 seconds
Chunk 7/348 inserted (10000 rows) in 0.55 seconds
Chunk 8/348 inserted (10000 rows) in 0.57 seconds
Chunk 9/348 inserted (10000 rows) in 0.60 seconds
Chunk 10/348 inserted (10000 rows) in 0.57 seconds
Chunk 11/348 inserted (10000 rows) in 0.61 seconds
Chunk 12/348 inserted (10000 rows) in 0.56 seconds
Chunk 13/348 inserted (10000 rows) in 0.56 seconds
Chunk 14/348 inserted (10000 rows) in 0.64 seconds
Chunk 15/348 inserted (10000 rows) in 0.55 seconds
Chunk 16/348 inserted (10000 rows) in 0.54 seconds
Chunk 17/348 inserted (10000 rows) in 0.58 seconds
Chunk 18/348 inserted (10000 rows) in 0.55 seconds
Chunk 19/348 inserted (10000 rows) in 0.57 seconds
Chunk 20/348 inserted (10000 rows) in 0.

In [7]:
# -----------------------------
# Étape 7 : Vérifier le nombre de lignes importées
# -----------------------------
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("SELECT COUNT(*) FROM yellow_tripdata_2025_01;"))
    print(f"Nombre de lignes importées : {result.scalar()}")


Nombre de lignes importées : 3475226
